# Exercise 4 - Nested Parallelism

**GOAL:** The goal of this exercise is to show how to create nested tasks by calling a remote function inside of another remote function.

In this exercise, you will implement the structure of a parallel hyperparameter sweep which trains a number of models in parallel. Each model will be trained using parallel gradient computations.

### Concepts for this Exercise - Nested Remote Functions

Remote functions can call other functions. For example, consider the following.

```python
@ray.remote
def f():
    return 1

@ray.remote
def g():
    # Call f 4 times and return the resulting object IDs.
    return [f.remote() for _ in range(4)]

@ray.remote
def h():
    # Call f 4 times, block until those 4 tasks finish,
    # retrieve the results, and return the values.
    return ray.get([f.remote() for _ in range(4)])
```

Then calling `g` and `h` produces the following behavior.

```python
>>> ray.get(g.remote())
[ObjectID(b1457ba0911ae84989aae86f89409e953dd9a80e),
 ObjectID(7c14a1d13a56d8dc01e800761a66f09201104275),
 ObjectID(99763728ffc1a2c0766a2000ebabded52514e9a6),
 ObjectID(9c2f372e1933b04b2936bb6f58161285829b9914)]

>>> ray.get(h.remote())
[1, 1, 1, 1]
```

**One limitation** is that the definition of `f` must come before the definitions of `g` and `h` because as soon as `g` is defined, it will be pickled and shipped to the workers, and so if `f` hasn't been defined yet, the definition will be incomplete.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import time

In [2]:
ray.init(num_cpus=9, redirect_output=True)

Waiting for redis server at 127.0.0.1:19479 to respond...
Waiting for redis server at 127.0.0.1:18958 to respond...
Starting local scheduler with 9 CPUs, 0 GPUs

View the web UI at http://localhost:8889/notebooks/ray_ui42278.ipynb?token=3ea9506cf13535c7dd7420da299adb9f44642fc1d4ad87df



{'local_scheduler_socket_names': ['/tmp/scheduler2216096'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store8375638', manager_name='/tmp/plasma_manager65974989', manager_port=62709)],
 'redis_address': '127.0.0.1:19479',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui42278.ipynb?token=3ea9506cf13535c7dd7420da299adb9f44642fc1d4ad87df'}

This example represents a hyperparameter sweep in which multiple models are trained in parallel. Each model training task also performs data parallel gradient computations.

**EXERCISE:** Turn `compute_gradient` and `train_model` into remote functions so that they can be executed in parallel. Inside of `train_model`, do the calls to `compute_gradient` in parallel and fetch the results using `ray.get`.

In [3]:
@ray.remote
def compute_gradient(data):
    time.sleep(0.03)
    return 1

@ray.remote
def train_model(hyperparameters):
    result = 0
    for i in range(10):
        # EXERCISE: After you turn "compute_gradient" into a remote function,
        # you will need to call it with ".remote". The results must be retrieved
        # with "ray.get" before "sum" is called.
        result += sum(ray.get([compute_gradient.remote(j) for j in range(2)]))
    return result

**EXERCISE:** The code below runs 3 hyperparameter experiments. Change this to run the experiments in parallel.

In [4]:
%%time
# Sleep a little to improve the accuracy of the timing measurements below.
time.sleep(2.0)
start_time = time.time()

# Run some hyperparaameter experiments.
results = []
for hyperparameters in [{'learning_rate': 1e-1, 'batch_size': 100},
                        {'learning_rate': 1e-2, 'batch_size': 100},
                        {'learning_rate': 1e-3, 'batch_size': 100}]:
    results.append(train_model.remote(hyperparameters))

results = [ray.get(i) for i in results]
end_time = time.time()
duration = end_time - start_time

CPU times: user 1.87 ms, sys: 1.21 ms, total: 3.07 ms
Wall time: 2.34 s


In [5]:
results, duration

([20, 20, 20], 0.34412097930908203)

**VERIFY:** Run some checks to verify that the changes you made to the code were correct. Some of the checks should fail when you initially run the cells. After completing the exercises, the checks should pass.

In [6]:
assert results == [20, 20, 20]
assert duration < 0.5, ('The experiments ran in {} seconds. This is too '
                         'slow.'.format(duration))
assert duration > 0.3, ('The experiments ran in {} seconds. This is too '
                        'fast.'.format(duration))

print('Success! The example took {} seconds.'.format(duration))

Success! The example took 0.34412097930908203 seconds.


**EXERCISE:** Use the UI to view the task timeline and to verify that the pattern makes sense.

In [20]:
import ray.experimental.ui as ui
ui.task_timeline()